In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path
import time
import tensorflow as tf
import numpy as np

In [5]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('num_epochs', 5, 'Number of epochs to run trainer.')
flags.DEFINE_integer('batch_size', 4096, 'Batch size.')
flags.DEFINE_string('train_dir', '/data', 'Directory with the training data.')

In [6]:
TRAIN_FILE = "tf-records.data"
VAL_FILE = ""

In [142]:
def extract_features(serialized_example):
    """
    Extracts a `dict` of named features from the serialized `tf.train.Example`
    """
    return tf.parse_single_example(
        serialized=serialized_example,
        features={
            'label': tf.FixedLenFeature([99], dtype=tf.int64),
            'input': tf.FixedLenFeature([20,111], dtype=tf.int64),
        }
    )

def deserialize_example(serialized_example):
    """
    Converts a serialized `tf.train.Example` to FP32 Tensors
    """
    features = extract_features(serialized_example)
    input = tf.cast(features['input'], tf.float32)
    label = tf.cast(features['label'], tf.float32)
    return input, label

def read_and_decode(filename_queue):
    """
    Read and Deserialize a single `tf.train.Example` from a TFRecord file.
    """
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    return deserialize_example(serialized_example)

In [165]:
def get_batch(batch_size=None, num_epochs=None, train=True):
    """
    Read in shuffled `inputs` and `labels` from either train or val files.
    
    Returns:
        `inputs` : [batch_size, 20, 111]
        `labels` : [batch_size, 99]
    """
    batch_size = batch_size or 128
    num_epochs = num_epochs or 1
    
    filename = os.path.join(FLAGS.train_dir, TRAIN_FILE if train else VALIDATION_FILE)
    
    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs)
        input, label = read_and_decode(filename_queue)
        inputs, labels = tf.train.shuffle_batch(
            [input, label], batch_size=batch_size, num_threads=4,
            capacity=3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=batch_size
        )
        return inputs, labels

## Test Reading Input

The following code block reads the first batch of input data and print out the details.

In [185]:
def make_example_new(seq, label):
    x = np.transpose(np.nonzero(seq))
    y = np.transpose(np.nonzero(label)).reshape(-1)
    return tf.train.Example(features=tf.train.Features(feature={
        'input_dense_dimensions': tf.train.Feature(int64_list=tf.train.Int64List(value=seq.shape)),
        'sparse_index_dimensions': tf.train.Feature(int64_list=tf.train.Int64List(value=x.shape)),
        'input': tf.train.Feature(int64_list=tf.train.Int64List(value=x.astype(int).reshape(-1))),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=y)),
        'label_length': tf.train.Feature(int64_list=tf.train.Int64List(value=[len(label)])),
    }))

def extract_features_new(serialized_example):
    """
    Extracts a `dict` of named features from the serialized `tf.train.Example`
    """
    return tf.parse_single_example(
        serialized=serialized_example,
        features={
            'input_dense_dimensions': tf.FixedLenFeature([2], dtype=tf.int64),
            'sparse_index_dimensions': tf.FixedLenFeature([2], dtype=tf.int64),
            'input': tf.FixedLenFeature([80], dtype=tf.int64),
            'label': tf.FixedLenFeature([1], dtype=tf.int64),
            'label_length': tf.FixedLenFeature([1], dtype=tf.int64),
        }
    )

def deserialize_example_new(serialized_example):
    """
    Converts a serialized `tf.train.Example` to FP32 Tensors
    """
    features = extract_features_new(serialized_example)
    shape_sparse = tf.cast(features['sparse_index_dimensions'], tf.int32)
    indices = tf.reshape(tf.cast(features['input'], tf.int32), shape_sparse)
    values = tf.ones([shape_sparse[0]])
    shape_dense = tf.cast(features['input_dense_dimensions'], tf.int32)
    input = tf.sparse_to_dense(indices, shape_dense, values)
    label = tf.one_hot(features['label'][0], 99, on_value=1., off_value=0., dtype=tf.float32)
    return input, label

In [188]:
def process_batch(input, label, iteration):
    with open("data/ornl_{0}".format(iteration), "w") as output:
        writer = tf.python_io.TFRecordWriter(output.name)
        for _x, _y in itertools.izip(_input, _label):
            example = make_example_new(_x, _y)
            writer.write(example.SerializeToString())
        

In [190]:
import numpy as np
import itertools

batch_size = 4096

with tf.Session() as sess:
    input, label = get_batch(batch_size=batch_size, num_epochs=1)
    init_op = tf.initialize_all_variables()
    local_op = tf.initialize_local_variables()
    sess.run(init_op)
    sess.run(local_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    try:
        
        
        iteration = 0
        while not coord.should_stop():
            _input, _label = sess.run([input, label])
            process_batch(_input, _label, iteration)
            iteration = iteration + 1
                
                #example = make_example_new(_x, _y)
                #mock_input = tf.constant(example.SerializeToString(), dtype=tf.string)
                #xn, yn = deserialize_example_new(mock_input)
                #_xn, _yn = sess.run([xn,yn])
                #assert np.array_equal(_x, _xn)
                #assert np.array_equal(_y, _yn)
            
            
        
        
        
    except tf.errors.OutOfRangeError, e:
        print("epoch limit reached")
        coord.request_stop(e)
    finally:
        coord.request_stop()
        coord.join(threads)

epoch limit reached


In [ ]:
#_tmp1 = numpy.transpose(numpy.nonzero(_input[0]))
        #print(tmp1)
        #print(_tmp1.shape)
        
        #tmp1 = tf.constant(_tmp1)
        #values = tf.ones([tmp1.get_shape()[0]])
        #s2d = tf.sparse_to_dense(tmp1, (20, 111), values)
        #s2d = tf.SparseTensor(indices=tmp1, values=tf.ones([tmp1.get_shape()[0]]), shape=(20,111))
        #_s2d = sess.run(s2d)
        
        #print(_s2d.shape)
        #print(_s2d)
        """
        _features, _class = tmp1[0::2], tmp1[1::2]
        depth = len(_features)
        _features = numpy.transpose(_features)[1]
        _class = numpy.transpose(_class)[1][0]
        #print(_features, _class)
        
        # expand class to length of features
        _cls = numpy.array(_class, ndmin=len(_features), copy=False)
        fv = tf.constant(_features)
        cl = tf.constant(_cls)
        _i = tf.cast(tf.one_hot(fv, 99, on_value=1, off_value=0, axis=-1, dtype=tf.int32), tf.float32)
        _c = tf.cast(tf.one_hot(cl, 12, on_value=1, off_value=0, axis=-1, dtype=tf.int32), tf.float32)
        #_z = tf.concat(1,[_i,_c])
        _t, _tt = sess.run([_i, _c])
        print(_t)
        
        print(_t.shape)
        print(_tt.shape)
        """
        #print(numpy.array_equal(_s2d, _input[0]))

In [6]:
with tf.Session() as sess:
    input, label = get_batch(batch_size=1024, num_epochs=1)
    #filename = os.path.join(FLAGS.train_dir, TRAIN_FILE if True else VALIDATION_FILE)
    #filename_queue = tf.train.string_input_producer([filename], num_epochs=1)
    #input, label = read_and_decode(filename_queue)
    init_op = tf.initialize_all_variables()
    local_op = tf.initialize_local_variables()
    sess.run(init_op)
    sess.run(local_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    try:
        step = 0
        while not coord.should_stop():
            
            _inputs, _labels = sess.run([input, label])
            
            

            # Print an overview fairly often.
            if step % 50 == 0:
                print(step, loss_value, duration)
    
            step += 1
            
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (10, step))
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

with open("tf-records.data", "w") as output:
    writer = tf.python_io.TFRecordWriter(output.name)
    for input, label in itertools.izip(X,y):
        writer.write(make_example(input, label).SerializeToString())
    writer.close()
        
    # Wait for threads to finish.
    coord.join(threads)
    
    try:
        sess.run([input, label])
        print(input)
        print(label)
        print(label.eval())
        sess.run([input, label])
        print(input)
        print(label)
        print(label.eval())
    except OutOfRangeError, e:
        print("out of range")
        coord.request_stop(e)
    finally:
        coord.request_stop()
        coord.join(threads)

Tensor("input/shuffle_batch:0", shape=(4, 20, 111), dtype=float32)
Tensor("input/shuffle_batch:1", shape=(4, 99), dtype=float32)
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

## Train the ORNL model

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.objectives import categorical_crossentropy

Using TensorFlow backend.


In [8]:
rnn_size = 50
seq_length = 20
num_vocab = 99
num_classes = 12

batch_size=2048
num_epochs=10

with tf.name_scope("nn"):
    model = Sequential()
    model.add( LSTM(rnn_size, input_shape=(seq_length, num_vocab + num_classes)) )
    model.add( Dense(num_vocab) )
    model.add( Activation('softmax') )

def tower_loss(scope):
    """
    Calculate the total loss for a given tower with its own batch of inputs
    """
    inputs, labels = get_batch(batch_size=batch_size, num_epochs=num_epochs)    
    logits = model(inputs)
    losses = tf.reduce_mean(categorical_crossentropy(labels, logits))
    
    #with tf.control_dependencies([loss_averages_op]):
    #    total_loss = tf.identity(total_loss)

    return losses

train_step = tf.train.RMSPropOptimizer(0.0005).minimize(loss)

In [10]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    keras.backend.set_session(sess)
    
    init_op = tf.group(tf.initialize_all_variables(),
                       tf.initialize_local_variables())
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        step = 0
        while not coord.should_stop():
            start_time = time.time()
            
            _, loss_value = sess.run([train_step, loss]) #, feed_dict={
            #        inputs: batch[0].eval(),
            #        labels: batch[1].eval()
            #})
        
            duration = time.time() - start_time

            # Print an overview fairly often.
            if step % 50 == 0:
                print(step, loss_value, duration)
    
            step += 1
            
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (10, step))
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

    # Wait for threads to finish.
    coord.join(threads)

0 4.57692 1.60616993904
50 4.57074 0.364095926285
100 4.44722 0.364655017853
150 3.9249 0.349284887314
200 3.65361 0.361363172531
250 3.42057 0.352455854416
300 3.5263 0.394969940186
350 3.59692 0.371726036072
400 3.3307 0.389976978302
450 3.4354 0.360405921936
500 3.21467 0.371031045914
550 3.38876 0.421336889267
600 3.36976 0.360926866531
650 3.22834 0.397493839264
700 3.20966 0.378099918365
750 3.19034 0.359433889389
800 3.16402 0.370951890945
850 3.09722 0.356650829315
900 3.02577 0.363167047501
950 3.09209 0.376534938812
1000 3.0116 0.36333489418
1050 3.00987 0.369508981705
1100 2.83554 0.385227918625
1150 2.89974 0.359158039093
1200 2.94319 0.354926109314
1250 3.06177 0.366718053818
1300 2.95086 0.358301877975
1350 2.82427 0.357882976532
1400 2.735 0.389733076096
1450 2.75951 0.374725103378
1500 2.91092 0.366447925568
1550 2.77232 0.362352132797
1600 2.87021 0.363610029221
1650 2.62245 0.393249988556
1700 2.6608 0.414990901947
1750 2.79919 0.362905025482
1800 2.71966 0.4077911376